In [2]:
import pandas as pd
import numpy as np
from clustering import get_cluster_data, generate_validation_plots
from clustering import clustering_experiment, generate_cluster_plots
from clustering import generate_component_plots
from helpers import get_abspath, save_array
from sklearn.decomposition import FastICA
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn

winepath = get_abspath('winequality.csv', 'data/experiments')
seismicpath = get_abspath('seismic_bumps.csv', 'data/experiments')
wine = np.loadtxt(winepath, delimiter=',')
seismic = np.loadtxt(seismicpath, delimiter=',')

# split data into X and y
wX = wine[:, :-1]
wY = wine[:, -1]
sX = seismic[:, :-1]
sY = seismic[:, -1]

In [33]:
ica = FastICA(random_state=0)
dims = range(1, wX.shape[1] + 1)

df = None
kurt = {}
for dim in dims:
    ica.set_params(n_components=dim)
    tmp = ica.fit_transform(wX)
    df = pd.DataFrame(tmp)
    df = df.kurt(axis=0)
    kurt[dim] = df.abs().mean()
    
#     print dim, df.abs().mean()

In [40]:
res = pd.DataFrame.from_dict(kurt, orient='index')

In [46]:
res.rename(columns={0:'kurtosis', inplace=True})

,df
1,0.182839
2,0.598654
3,1.556014
4,2.198153
5,8.752726
6,7.981766
7,8.605328
8,12.683083
9,12.110247
10,12.614970


In [13]:
for dim in dims:
    print dim

1
2
3
4
5
6
7
8
9
10
11
12
